In [20]:
# 최대전력 데이터 모으기
import pandas as pd
df = pd.DataFrame()
for i in range(1,25):   #2009년7월,8월~2020년7월8월
    df1 = pd.read_excel(f'dataset/최대전력/{i}.xls')
    df1 = df1[::-1]
    df2 = df.append(df1)
    df = df2

In [21]:
# 중간에 날자가아닌 다른값이 들어간, null값 삭제.
df = df[df['일시'].notnull()]
#최대전력만 가져오기
df = df[['일시','최대전력(만kW)']]
df = df.rename({'일시':'date','최대전력(만kW)':'금년'},axis =1)
df = df.reset_index()
df = df.drop('index',axis=1)

In [22]:
date = pd.DataFrame()
for i in [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]:
    d = pd.date_range(start = f'{i}-07-01', end =  f'{i}-08-31', freq = 'D')
    d = pd.DataFrame(d)
    date1 = date.append(d)
    date = date1
date = date.reset_index()
date = date.drop('index',axis=1)
df['date'] = date

In [23]:
df = df.set_index('date')

In [24]:
df.head()  #최대전력데이터

,금년
date,
2009-07-01,"5,446.5"
2009-07-02,"5,508.1"
2009-07-03,"5,739.8"
2009-07-04,"5,135.6"
2009-07-05,"4,853.8"


In [26]:
#점유율
use = pd.DataFrame()
for i in range(1,13):  
    use_read = pd.read_excel(f'dataset/점유율/{i}.xlsx')
    use_read = use_read.iloc[28:30]  #7,8월 점유율 추출
    use1 = use.append(use_read)
    use = use1

In [27]:
use = use[['Unnamed: 2']]
li = []
for i in [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]:
    i7 = f'{i}-07-01'
    i8 = f'{i}-08-01'
    li.append(i7)
    li.append(i8)
use['date'] = li
# use.to_csv('data/점유율0708.csv')
use.date = use.date.astype('datetime64')
use = use.set_index('date')

C:\anaconda3\envs\cakd3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [28]:
use = use.rename(columns = {'Unnamed: 2' : '가정용점유율(%)'})

In [29]:
use.head()

,가정용점유율(%)
date,
2009-07-01,14.2
2009-08-01,15.2
2010-07-01,13.8
2010-08-01,15.4
2011-07-01,13.2


In [30]:
all_data = df.join(use, how='left')
all_data = all_data.fillna( method = 'ffill')

In [31]:
all_data['금년'] = all_data['금년'].str.replace(',','')
all_data['금년'] = all_data['금년'].astype('float')
all_data['가정용점유율(%)'] = all_data['가정용점유율(%)'].astype('float')
all_data['가정용전력'] = all_data['금년']*all_data['가정용점유율(%)']/100

In [32]:
all_data.head(5)

,금년,가정용점유율(%),가정용전력
date,,,
2009-07-01,5446.5,14.2,773.4030
2009-07-02,5508.1,14.2,782.1502
2009-07-03,5739.8,14.2,815.0516
2009-07-04,5135.6,14.2,729.2552
2009-07-05,4853.8,14.2,689.2396


In [33]:
all_data.to_excel('dataset/가정전력.xlsx')